In [ ]:
#@title set global env

import os
os.environ["OPENAI_API_KEY"] =
os.environ["ANTHROPIC_API_KEY"] =

In [ ]:
#@title import, functions
# set up
!apt install jpegoptim imagemagick
!pip install anthropic
import math
import csv
import os
import base64
import requests
import json
import re
import anthropic
import httpx
import pandas as pd




def extract_coordinates_response(text):
    text = text.replace('*', '')
    text = text.replace('° N', '')
    text = text.replace('° E', '')
    text = text.replace('° S', '')
    text = text.replace('° W', '')
    pattern = re.compile(r"Coordinates:\s*(\-?\d+\.\d+),\s*(\-?\d+\.\d+)")
    match = pattern.search(text)
    if match:
        # Extract the latitude and longitude as floats
        latitude = match.group(1)
        longitude = match.group(2)
        return f"{latitude}, {longitude}"
    else:
        return None


def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def degrees2radians(degrees):
    return degrees * (math.pi/180)

def distance(lat1, lon1, lat2, lon2):
    earthRadiusKm = 6371;

    dLat = degrees2radians(lat2-lat1);
    dLon = degrees2radians(lon2-lon1);
    lat1 = degrees2radians(lat1);
    lat2 = degrees2radians(lat2);
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.sin(dLon/2) * math.sin(dLon/2) * math.cos(lat1) * math.cos(lat2);
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a));
    distance = earthRadiusKm * c;
    return distance

def extract_coordinates_filename(filename):
    name, _ = os.path.splitext(filename)
    lat, lon = name.split('_')
    return f"{lat}, {lon}"

def response_openai(api_key, prompt, filename):
    path = filename
    base64_image = encode_image(path)

    headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }

        # Construct the payload
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{prompt}"
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 1000
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()



def response_claude(prompt, filename):
    client = anthropic.Anthropic()

    image1_media_type = "image/jpeg"
    path = filename # 5MB limit
    base64_image = encode_image(path)

    message = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": image1_media_type,
                            "data": base64_image,
                        },
                    },
                    {
                        "type": "text",
                        "text": f"{prompt}"
                    }
                ],
            }
        ],
    )
    # get content with message.content[0].text
    return message

def grade(prompt, response):
    headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }

        # Construct the payload
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"{prompt} {response}"
                    },
                ]
            }
        ],
        "max_tokens": 500
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    print(response.json())
    return response.json()

def get_criteria(text):
    pattern = r"climate: (\d), architecture: (\d), street_signs: (\d), language: (\d), landmark: (\d), vegetation: (\d), vehicle: (\d), urban_layout: (\d), cultural_element: (\d)"

    matches = re.findall(pattern, text)

    if matches:
        result = {
            "climate": int(matches[0][0]),
            "architecture": int(matches[0][1]),
            "street_signs": int(matches[0][2]),
            "language": int(matches[0][3]),
            "landmark": int(matches[0][4]),
            "vegetation": int(matches[0][5]),
            "vehicle": int(matches[0][6]),
            "urban_layout": int(matches[0][7]),
            "cultural_element": int(matches[0][8])
        }
        return result
    else:
        return None



In [ ]:
#@title downloading dataset and compressing images
!wget link/to/dataset -O /content/dataset.zip
!unzip -FF /content/dataset.zip -d /content/dataset/
!cd /content/dataset && mogrify -resize 50% *.jpg && jpegoptim --size=5000K --strip-all *.jpg

In [ ]:
#@title INITIALIZE CSV FILE HERE
image_directory = 'dataset'

# INITIALIZE CSV FILE
output_csv_path = 'temp.csv'

file_list = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

with open(output_csv_path, mode='w', newline='') as file:
    csv_writer = csv.writer(file)
    # 'climate','architecture','street_signs', 'language', 'landmark', 'vegetation', 'vehicle', 'urban_layout', 'cultural_element'
    csv_writer.writerow(['source','actual_coordinates','gpt_response', 'gpt_guess', 'gpt_distance','gpt_grade','claude_response', 'claude_guess', 'claude_distance', 'claude_grade'])
    for filename in file_list:
        # print(filename)
        coordinate = extract_coordinates_filename(filename)
        csv_writer.writerow([filename, coordinate] + [''] * 8)
        print(f'Added {filename} with coordinate {coordinate}')


In [ ]:
#@title gpt
csv_file_path = 'temp.csv'
api_key = os.environ["OPENAI_API_KEY"]
prompt = "You are a world class GeoGuessr player, and can guess the location of images very well based on the images alone. Here is an image, try your best to guess the location. Make a chain of thought to infer clues from details in the image. Then, using the information, give a best guess to the coordinates of the image. Be very specific, don't give a general coordinates of the city or country, but infer what area the image is from. The coordinates must follow `Coordinates:` (on the same line) and be surrounded by asterisks like `lat, long`. Use plain text, no markdown."

# Load the CSV into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Iterate over the rows in the DataFrame


for index, row in df.iterrows():
    print(index, row['source'])
    path_to_image = row['source']
    response = response_openai(os.environ["OPENAI_API_KEY"], prompt, "/content/dataset/" + path_to_image)
    print(response)

    # Store the response content
    gpt_response = response['choices'][0]['message']['content']
    df.at[index, 'gpt_response'] = gpt_response

    # Extract coordinates from the response
    gpt_guess = extract_coordinates_response(gpt_response)
    df.at[index, 'gpt_guess'] = gpt_guess

    print(f"gpt guess: {gpt_guess}")

    # Parse actual coordinates from the DataFrame
    lat0, lon0 = map(float, df.at[index, 'actual_coordinates'].split(','))

    # Parse guessed coordinates from the DataFrame
    lat1, lon1 = map(float, gpt_guess.split(','))

    # Calculate the distance between actual and guessed coordinates
    gpt_distance = distance(lat0, lon0, lat1, lon1)
    df.at[index, 'gpt_distance'] = gpt_distance

    print(f"Distance: {gpt_distance}")

df.to_csv(csv_file_path, index=False)


In [ ]:
#@title gpt grade
df = pd.read_csv(csv_file_path)

grader_prompt = "You are a professional AI response grader and can accurately tag the reponses with 'climate','architecture','street_signs', 'language', 'landmark', 'vegetation', 'vehicle', 'urban_layout', 'cultural_element'. Answer whether the given response infers the location from these information, give them 0 for no or 1 for yes. Your response should be like the following ```climate: 1, architecture: 0, street_element: 1, langaage: 1, landmark: 1, vegetation: 1, vehicle:0, vehicle:1, urban_layout:1, cultural_element:0``` \nThe response: "

# Iterate over the rows in the DataFrame
for index, row in df.iterrows():
    response = df.at[index, "gpt_response"]
    grades = grade(grader_prompt, response)
    print(grades)
    criterias = get_criteria(grades['choices'][0]['message']['content'])
    print(criterias)
    df.at[index, 'gpt_grade'] = json.dumps(criterias)
# Save the updated DataFrame back to a CSV file
df.to_csv(csv_file_path, index=False)


In [ ]:
#@title claude
csv_file_path = 'temp.csv'
prompt = "You are a world class GeoGuessr player, and can guess the location of images very well based on the images alone. Here is an image, try your best to guess the location. Make a chain of thought to infer clues from details in the image. Then, using the information, give a best guess to the coordinates of the image. Be very specific, don't give a general coordinates of the city or country, but infer what area the image is from. The coordinates must follow this style `Coordinates: lat, long`. Use plain text, no markdown. Your response must always contain the coordinates"

# Load the CSV into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Iterate over the rows in the DataFrame
for index, row in df.iterrows():
    # print(row['source'])
    path_to_image = row['source']
    response = response_claude(prompt, "/content/dataset/" + path_to_image)
    # print(response)
    df.at[index, 'claude_response'] = response.content[0].text
    df.at[index, 'claude_guess'] = extract_coordinates_response(df.at[index, 'claude_response'])

    lat0, lon0 = df.at[index, 'actual_coordinates'].split(',')
    lat0 = float(lat0)
    lon0 = float(lon0)

    # print(row['claude_guess'])
    # print(df.at[index, 'claude_response'])
    lat1, lon1 = df.at[1, 'claude_guess'].split(',')
    lat1 = float(lat1)
    lon1 = float(lon1)
    print(f"{lat0}, {lon0}, {lat1}, {lon1}")

    df.at[index, 'claude_distance'] = distance(lat0, lon0, lat1, lon1)
    print(df.at[index, 'claude_distance'])

df.to_csv(csv_file_path, index=False)


In [ ]:
#@title claude grade
df = pd.read_csv(csv_file_path)

grader_prompt = "You are a professional AI response grader and can accurately tag the reponses with 'climate','architecture','street_signs', 'language', 'landmark', 'vegetation', 'vehicle', 'urban_layout', 'cultural_element'. Answer whether the given response infers the location from these information, give them 0 for no or 1 for yes. Your response should be like the following ```climate: 1, architecture: 0, street_element: 1, langaage: 1, landmark: 1, vegetation: 1, vehicle:0, vehicle:1, urban_layout:1, cultural_element:0``` \nThe response: "

# Iterate over the rows in the DataFrame
for index, row in df.iterrows():
    response = df.at[index, "claude_response"]
    grades = grade(grader_prompt, response)
    print(grades)
    criterias = get_criteria(grades['choices'][0]['message']['content'])
    print(criterias)
    df.at[index, 'claude_grade'] = json.dumps(criterias)
# Save the updated DataFrame back to a CSV file
df.to_csv(csv_file_path, index=False)
